In [ ]:
#r "BoSSSpad.dll"
// #r ".\binaries\XNSEC.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
//#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Unable to open database: 
primary path: d:\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #0 at 'd:\bosss_db_Natconv': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST
No alternative paths specified.
IOException caught while opening database #1 at '\\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #2 at '\\hpccluster\hpccluste

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#27.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
using BoSSS.Application.XNSEC;

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedCavity_ConvergenceStudy");

Project name is set to 'HeatedCavity_ConvergenceStudy'.
NotSupportedException caught during creation/opening of default database: `AllowedDatabasesPaths` not specified, unable to create Database (should be specified in ~/.BoSSS/etc/BatchProcessorConfig.json)..


In [ ]:
int NCellsFinestMesh = (int) Math.Pow(Math.Pow(2,7+1),2); 

In [ ]:
var myBatch = BoSSSshell.GetDefaultQueue();
var myDb = myBatch.CreateOrOpenCompatibleDatabase("HeatedCavity_ConvergenceStudy");


Opening existing database 'C:\Databases\HeatedCavity_ConvergenceStudy'.


In [ ]:
myDb.Path

C:\Databases\HeatedCavity_ConvergenceStudy

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
BoSSSshell.WorkflowMgm.hConvergence.Update();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable; // The session table has to be created again in order to see the results from hconvergence

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Temperature","Grid:hMin","Grid:NoOfCells" ,"L2Error_Temperature");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=NCellsFinestMesh );
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_Temperature", "DGdegree:Temperature");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;
ErrorPlot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Temperature3 
 
 
 DGdegree:Temperature3 
 
 
 
 
 
 
 
 
 DGdegree:Temperature2 
 
 
 DGdegree:Temperature2 
 
 
 
 
 
 
 
 
 DGdegree:Temperature1 
 
 
 DGdegree:Temperature1

In [ ]:
ErrorPlot.Regression()

index,Key,Value
0,DGdegree:Temperature3,4.8020048460418
1,DGdegree:Temperature2,3.9304986124824492
2,DGdegree:Temperature1,1.883114377668935


In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2Error_VelocityX");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=NCellsFinestMesh );
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_VelocityX", "DGdegree:Velocity*");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;
ErrorPlot.PlotNow()     // No semicolon!!

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1

In [ ]:
var SlopesVelX = ErrorPlot.Regression();

In [ ]:
SlopesVelX.Pick(0).Value 

3.659456757406345

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2Error_VelocityY");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=NCellsFinestMesh );
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_VelocityY", "DGdegree:Velocity*");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;
ErrorPlot.PlotNow()     // No semicolon!!

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1

In [ ]:
ErrorPlot.Regression()

index,Key,Value
0,DGdegree:Velocity*3,3.8466780298791985
1,DGdegree:Velocity*2,3.497448478980067
2,DGdegree:Velocity*1,1.9267843475721205


In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Pressure","Grid:hMin","Grid:NoOfCells" ,"L2Error_Pressure");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=NCellsFinestMesh );
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_Pressure", "DGdegree:Pressure");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;
ErrorPlot.PlotNow()     // No semicolon!!

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure2 
 
 
 DGdegree:Pressure2 
 
 
 
 
 
 
 
 
 DGdegree:Pressure1 
 
 
 DGdegree:Pressure1 
 
 
 
 
 
 
 
 
 DGdegree:Pressure0 
 
 
 DGdegree:Pressure0

In [ ]:
ErrorPlot.Regression()

index,Key,Value
0,DGdegree:Pressure2,-0.379666437916206
1,DGdegree:Pressure1,2.203109085407051
2,DGdegree:Pressure0,-0.2421016829868629
